In [ ]:
'''
this part we have done the exact same thing as UniqueValues except this time insted of using 20 Lakh metapaths we use 
top 2 Lakh and recalcute all the vaules and person co-efficent with respect to Word|Word metapath as our label.

''' 

In [ ]:
import pandas as pd
import sqlite3
import pickle
import json
from collections import defaultdict
import threading
from multiprocessing.managers import BaseManager, DictProxy
import multiprocessing

In [ ]:
conn = sqlite3.connect("Extracted.db")
conn1=sqlite3.connect("Features.db")

In [ ]:
#During Multiprocesing as Python is built on C, defaultdict is not defined so we are formally defining it
class MyManager(BaseManager):
    pass

MyManager.register('defaultdict', defaultdict, DictProxy)
#registering defualtdict into our manager

In [ ]:
mgr = MyManager()
mgr.start()
features = mgr.defaultdict(int)



In [ ]:
sample = pd.read_sql("SELECT * FROM SampleZero", conn)
#we dont have any metapaths of length 2 after initial stages of filtering
metapath3 = pd.read_sql("SELECT * FROM Meta3_filteredPearson", conn1)
metapath4 = pd.read_sql("SELECT * FROM Meta4_filteredPearson", conn1)

In [ ]:
with open('CNG_count.pickle', 'rb') as handle:
    CNG_count = pickle.load(handle)

In [ ]:
with open('CNGG_count.pickle', 'rb') as handle:
    CNGG_count = pickle.load(handle)

In [ ]:
with open('Word_count.pickle', 'rb') as handle:
    Word_count = pickle.load(handle)

In [ ]:
with open('Lemma_count.pickle', 'rb') as handle:
    Lemma_count = pickle.load(handle)

In [ ]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [ ]:
innerdict = merge_two_dicts(CNG_count, CNGG_count)
#we merge them for easier selection in the future
#as denominators in metapath3,4 are either count of CNG or CNG_Group

In [ ]:
def from_dict(Type_1, Type_2):
    with open(Type_1 + '|' + Type_2 + '.json', 'r') as fp:
        d = json.load(fp)
    return d

In [ ]:
graph = { 'LemmaLemma': from_dict('Lemma', 'Lemma'), 'LemmaWord' : from_dict('Lemma', 'Word'), 'LemmaCNG' : from_dict('Lemma', 'CNG'), 'LemmaCNG_Group' : from_dict('Lemma', 'CNG_Group'), 'WordLemma' : from_dict('Word', 'Lemma'), 'WordWord' : from_dict('Word', 'Word'), 'WordCNG' : from_dict('Word', 'CNG'), 'WordCNG_Group' : from_dict('Word', 'CNG_Group'), 'CNGLemma' : from_dict('CNG', 'Lemma'), 'CNGWord' : from_dict('CNG', 'Word'), 'CNGCNG' : from_dict('CNG', 'CNG'), 'CNGCNG_Group' : from_dict('CNG', 'CNG_Group'), 'CNG_GroupLemma' : from_dict('CNG_Group', 'Lemma'), 'CNG_GroupWord' : from_dict('CNG_Group', 'Word'), 'CNG_GroupCNG' : from_dict('CNG_Group', 'CNG'), 'CNG_GroupCNG_Group' : from_dict('CNG_Group', 'CNG_Group') } 


In [ ]:
CNG_Distinct = len(CNG_count.keys())

In [ ]:
#defining the label, which is Word|Word
label = []
for i,r in sample.iterrows():
    value1 = r['Entity_1']
    value2 = r['Entity_2'] 
    try:
        num = graph['WordWord'][value1 + '|' + value2]
    except:
        num = 0
    den = Word_count[value1]
    prob = (float(num) + 1) /(den + CNG_Distinct)
    label.append(prob)


In [ ]:
def mp3(features, metapath3, sample):
    for mp3i,mp3r in metapath3.iterrows():
        #iterating through all metapaths of size 3
        node1 = mp3r['node1']
        node2 = mp3r['node2']
        node3 = mp3r['node3']
        print(node1, node2, node3)        
        #since it is a metapath of size 3 we have only 3 nodes
        vector = []
        for i,r in sample.iterrows():
            den1 = 0
            den2 = 0
            if node1 == 'Lemma':
                value1 = r['Entity_1'].split("_")[0]
                den1 = Lemma_count[value1]
                part1 = 'Lemma'
            elif node1 == 'CNG':
                value1 = r['Entity_1'].split("_")[1]
                den1 = CNG_count[value1]
                part1 = 'CNG'
            else:
                value1 = r['Entity_1']
                den = Word_count[value1]
                part1 = 'Word'
            value2 = node2            
            try:
                x = int(value2)
                part2 = 'CNG'
            except:
                part2 = 'CNG_Group' 
            den2 = innerdict[node2]               
            if node3 == 'Lemma':
                value3 = r['Entity_2'].split("_")[0]
                part3 = 'Lemma'
            elif node3 == 'CNG':
                value3 = r['Entity_2'].split("_")[1]
                part3 = 'CNG'
            else:
                value3 = r['Entity_2']
                part3 = 'Word'
            #selecting from innerdict as it has counts of both CNG and CNG_Groups
            part12 = part1 + part2
            try:                    
                num12 = graph[part12][value1 + '|' + value2]
            except:
                num12 = 0
            prob12 = (float(num12) + 1)/(den1 + CNG_Distinct)                              
            part23 = part2 + part3
            try:
                num23 = graph[part23][value2 + '|' + value3]
            except:
                num23 = 0
            prob23 = (float(num23) + 1)/(den2 + CNG_Distinct)
            prob = prob12*prob23
            vector.append(prob)
        features[node1 + '|' + node2 + '|'+ node3]  = vector

In [ ]:
def mp4(features ,metapath4, sample):
    for mp4i,mp4r in metapath4.iterrows():
        #iterating through all metapaths of size 4
        node1 = mp4r['node1']
        node2 = mp4r['node2']
        node3 = mp4r['node3']
        node4 = mp4r['node4']
        #since it is a metapath of size 4 we have only 4 nodes
        vector = []
        for i,r in sample.iterrows():
            den1 = 0
            den2 = 0
            den3 = 0
            if node1 == 'Lemma':
                value1 = r['Entity_1'].split("_")[0]
                den1 = Lemma_count[value1]
                part1 = 'Lemma'
            elif node1 == 'CNG':
                value1 = r['Entity_1'].split("_")[1]
                den1 = CNG_count[value1]
                part1 = 'CNG'
            else:
                value1 = r['Entity_1']
                den1 = Word_count[value1]
                part1 = 'Word'
            value2 = node2
            try:
                x = int(value2)
                part2 = 'CNG'
            except:
                part2 = 'CNG_Group'
            value3 = node3
            try:
                x = int(value3)
                part3 = 'CNG'
            except:
                part3 = 'CNG_Group'       
            if node4 == 'Lemma':
                value4 = r['Entity_2'].split("_")[0]
                part4 = 'Lemma'
            elif node4 == 'CNG':
                value4 = r['Entity_2'].split("_")[1]
                part4 = 'CNG'
            else:
                value4 = r['Entity_2']
                part4 = 'Word'
            den2 = innerdict[node2]
            den3 = innerdict[node3]
            part12 = part1 + part2
            try:
                num12 = graph[part12][value1 + '|' + value2]
            except:
                num12 = 0
            prob12 = (float(num12) + 1)/(den1 + CNG_Distinct)
            part23 = part2 + part3
            try:
                num23 = graph[part23][value2 + '|' + value3]
            except:
                num23 = 0
            prob23 = (float(num23) + 1)/(den2 + CNG_Distinct)
            part34 = part3 + part4
            try:
                num34 = graph[part34][value3 + '|' + value4]
            except:
                num34 = 0
            prob34 = (float(num34) + 1)/(den3 + CNG_Distinct)
            prob = prob12*prob23*prob34
            vector.append(prob)
        features[node1 + '|' + node2 + '|'+ node3 + '|' +node4]  = vector


In [ ]:
#Threading begins here
#Calculate Length of Metapahts so that we can slpit them for threading 
lenmp3 = len(metapath3.node1)
lenmp4 = len(metapath4.node1)

In [ ]:
#Running a total of 40 processes on 6 servers, mp2 : 1 Process , mp3: 1 Process, mp4 : 40 Processes on 6 servers

serverno = 1
server = 0
#we change the value of server accroding to which server we are using 
step4 = int(lenmp4/(40*serverno))

ind4=[]

z = server*40*step4

#make sure to change range from 40 to 39, and include the commented line of code in last server.

for i in range(39):
    ind4.append([z, z+step4])
    z = z+step4
ind4.append([z,lenmp4-1])

print ind4

mgr = MyManager()
mgr.start()
#initalizing the manager and start it.

features = mgr.defaultdict(list)

#run mp3 only on one server

mp3(features ,metapath3, sample)

t1 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[0][0]:ind4[0][1], :], sample))
t2 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[1][0]:ind4[1][1], :], sample))
t3 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[2][0]:ind4[2][1], :], sample))
t4 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[3][0]:ind4[3][1], :], sample))
t5 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[4][0]:ind4[4][1], :], sample))
t6 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[5][0]:ind4[5][1], :], sample))
t7 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[6][0]:ind4[6][1], :], sample))
t8 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[7][0]:ind4[7][1], :], sample))
t9 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[8][0]:ind4[8][1], :], sample))
t10 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[9][0]:ind4[9][1], :], sample))
t11 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[10][0]:ind4[10][1], :], sample))
t12 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[11][0]:ind4[11][1], :], sample))
t13 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[12][0]:ind4[12][1], :], sample))
t14 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[13][0]:ind4[13][1], :], sample))
t15 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[14][0]:ind4[14][1], :], sample))
t16 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[15][0]:ind4[15][1], :], sample))
t17 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[16][0]:ind4[16][1], :], sample))
t18 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[17][0]:ind4[17][1], :], sample))
t19 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[18][0]:ind4[18][1], :], sample))
t20 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[19][0]:ind4[19][1], :], sample))
t21 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[20][0]:ind4[20][1], :], sample))
t22 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[21][0]:ind4[21][1], :], sample))
t23 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[22][0]:ind4[22][1], :], sample))
t24 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[23][0]:ind4[23][1], :], sample))
t25 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[24][0]:ind4[24][1], :], sample))
t26 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[25][0]:ind4[25][1], :], sample))
t27 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[26][0]:ind4[26][1], :], sample))
t28 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[27][0]:ind4[27][1], :], sample))
t29 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[28][0]:ind4[28][1], :], sample))
t30 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[29][0]:ind4[29][1], :], sample))
t31 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[30][0]:ind4[30][1], :], sample))
t32 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[31][0]:ind4[31][1], :], sample))
t33 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[32][0]:ind4[32][1], :], sample))
t34 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[33][0]:ind4[33][1], :], sample))
t35 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[34][0]:ind4[34][1], :], sample))
t36 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[35][0]:ind4[35][1], :], sample))
t37 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[36][0]:ind4[36][1], :], sample))
t38 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[37][0]:ind4[37][1], :], sample))
t39 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[38][0]:ind4[38][1], :], sample))
t40 = multiprocessing.Process(target=mp4, args=(features,metapath4.iloc[ind4[39][0]:ind4[39][1], :], sample))


t1.start()
t2.start()
t3.start()
t4.start()
t5.start()
t6.start()
t7.start()
t8.start()
t9.start()
t10.start()
t11.start()
t12.start()
t13.start()
t14.start()
t15.start()
t16.start()
t17.start()
t18.start()
t19.start()
t20.start()
t21.start()
t22.start()
t23.start()
t24.start()
t25.start()
t26.start()
t27.start()
t28.start()
t29.start()
t30.start()
t31.start()
t32.start()
t33.start()
t34.start()
t35.start()
t36.start()
t37.start()
t38.start()
t39.start()
t40.start()


t1.join()
t2.join()
t3.join()
t4.join()
t5.join()
t6.join()
t7.join()
t8.join()
t9.join()
t10.join()
t11.join()
t12.join()
t13.join()
t14.join()
t15.join()
t16.join()
t17.join()
t18.join()
t19.join()
t20.join()
t21.join()
t22.join()
t23.join()
t24.join()
t25.join()
t26.join()
t27.join()
t28.join()
t29.join()
t30.join()
t31.join()
t32.join()
t33.join()
t34.join()
t35.join()
t36.join()
t37.join()
t38.join()
t39.join()
t40.join()

#next time use pooling insted of writing so many processes
#tip : use ctrl + left mouse click to have cursor select multiple lines and change at once

In [ ]:
with open( 'PearsonFeatures.json', 'w') as fp:
    json.dump(features._getvalue(), fp)
mgr.join()
